In [ ]:
import sys
root_dir = '../../../'
sys.path.append(root_dir)

In [ ]:
import pickle
from matplotlib.lines import Line2D
from utils import *

In [ ]:
import warnings
warnings.filterwarnings("ignore")

## Load data

In [ ]:
lorenz = pd.read_csv(root_dir + '/data/lorenz.csv', header=None).iloc[:, 0].values
j = 150
X_train = lorenz[3_000 + j: 13_000 + j]
Y_true = lorenz[13_000 + j: 14_000 + j]

## Get patterns and motifs

In [ ]:
patterns = get_patterns(
    pattern_size=3, 
    max_distance=10, 
    patterns_percent=100
)

motifs = get_motifs(X_train, patterns)

print(len(motifs))
print(motifs[patterns[0]].shape)

## Algorithm

In [ ]:
trajectory_length = 50

trajectory, possible_predictions = daemon(
    trajectory_length = trajectory_length,
    X_train = X_train,
    Y_true = Y_true,
    motifs =  motifs,
    distance_eps = 0.005,
    cluster_eps = 0.01,
    min_samples = 5,
    cluster_1_proportion_threshold = -1,
    clusters_diff_threshold = -1,
    sigma = 0.0,
    return_possible_predictions = True,
    daemon_eps = 0.05
)

In [ ]:
plt.figure(figsize=(15, 5))
plot_Y_true(Y_true[:trajectory_length])
plot_predictions(trajectory)
plot_possible_predictions(possible_predictions)
plt.ylim(top=1.05)
plt.ylim(bottom=-0.05)
plt.xlabel('Time Step')
plt.ylabel('Time series value')
plt.legend(bbox_to_anchor=(1.25, 1))
plt.show()

In [ ]:
trajectory_length = 50
n = 96 * 10   # the number of iterations required to traverse Y_true

inputs = []
for i in range(n):
    _X_train = np.append(X_train, Y_true[:i])
    _Y_true = Y_true[i: i + trajectory_length]
    _input = (
        trajectory_length, 
        _X_train,
        _Y_true,
        motifs
    )   
    inputs.append(_input)


def task(x):
    trajectory_length, X_train, Y_true, motifs = x
    trajectory, possible_predictions = daemon(
        trajectory_length = trajectory_length,
        X_train = X_train,
        Y_true = Y_true,
        motifs =  motifs,
        distance_eps = 0.005,
        cluster_eps = 0.01,
        min_samples = 5,
        cluster_1_proportion_threshold = -1,
        clusters_diff_threshold = -1,
        sigma = 0.0,
        return_possible_predictions = True,
        daemon_eps = 0.05
    )
    return trajectory, possible_predictions


results = run_concurrently(task, inputs)

In [ ]:
with open('results-daemon-1.pickle', 'wb') as f:
    pickle.dump(results, f)